In [1]:
%pip install --upgrade pinecone-client pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install langchain_huggingface pydantic transformers

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.41
    Uninstalling langchain-core-0.2.41:
      Successfully uninstalled langchain-core-0.2.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.65 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.8 which is incompatible.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.3.8 which is incompatible.
langchain-cohere 0.1.9 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.3.8 which is incompatible.
langchain 0.2.16 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.8 which is incompatible.
langchain-community 0.2.17 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.8 which is incompatible.
langchain-text-splitters 0.2.4 requ

In [8]:
%pip install --upgrade langchain langchain_huggingface pydantic

  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.9 kB/s eta 0:00:00ta 0:00:17
Using cached langchain_text_splitters-0.3.0-py3-none-any.whl (25 kB)
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.4
    Uninstalling langchain-text-splitters-0.2.4:
      Successfully uninstalled langchain-text-splitters-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.16
    Uninstalling langchain-0.2.16:
      Successfully uninstalled langchain-0.2.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.67.1 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.3.2 which is incompatible.
langchain-community 0.2.17 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.3.2 which 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone , ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
pc = Pinecone(api_key=api_key)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )

/home/ayush/Code/ML-Projects/Gemini/Langchain/env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
index = pc.Index(index_name)

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

/home/ayush/Code/ML-Projects/Gemini/Langchain/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
from pinecone_text.sparse import BM25Encoder
bm25Encoder = BM25Encoder().default()
bm25Encoder


In [9]:
import nltk
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /home/ayush/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
sent = ["In 2023 , I visited Paris","In 2022 , I Visited New York","In 2021 , I visited New Orleans"]
bm25Encoder.fit(sent)
bm25Encoder.dump("bm25.json")
bm25Encoder = BM25Encoder().load("bm25.json")

100%|██████████| 3/3 [00:00<00:00, 4009.85it/s]


In [12]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25Encoder,index=index)

In [13]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x76c57529f170>, index=<pinecone.data.index.Index object at 0x76c6ab3fb530>)

In [14]:
retriever.add_texts(
    ["In 2023 , I visited Paris","In 2022 , I Visited New York","In 2021 , I visited New Orleans"]
)

100%|██████████| 1/1 [00:12<00:00, 12.10s/it]


In [16]:
retriever.invoke("in which year i had been to paris")

[Document(metadata={'score': 0.483954519}, page_content='In 2023 , I visited Paris'),
 Document(metadata={'score': 0.192793414}, page_content='In 2021 , I visited New Orleans'),
 Document(metadata={'score': 0.175562263}, page_content='In 2022 , I Visited New York')]